<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/Model_Bonet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [ ]:
# Define relevant variables for the ML task
START_EPOCH=0
NUM_EPOCHS=60
LR=0.0001
PATIENCE=2
BATCH_SIZE=20
NUM_WORKERS=0
NUM_GPUS=1
GPUS=0

In [ ]:
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [ ]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [ ]:
train_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.RandomAffine(
                                   20, translate=(0.2, 0.2),
                                   scale=(1, 1.2)),
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor()])

val_transform = transforms.Compose([transforms.Resize((500, 500)),
                               transforms.ToTensor()])

In [ ]:
#all_transforms = transforms.Compose([transforms.Resize((32,32)),
                                     #transforms.ToTensor(),
                                     #transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                                                          #std=[0.2023, 0.1994, 0.2010])
                                    # ])

In [ ]:
import torchvision

In [ ]:
# Create Training dataset
train_dataset = torchvision.datasets.ImageFolder(path ='/content/drive/My Drive/Project_Boneage/boneage_training_dataset/',transform = train_transform)

TypeError: ignored

In [ ]:
# Create Training dataset
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             transform = train_transform,
                                             download = True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Create Training dataset
train_dataset = torchvision.datasets.ImageNet(mpimg.imread(os.path.join(path,'boneage_training_dataset/'),
                                             transform = train_transform)

# Create Testing dataset
test_dataset = torchvision.datasets.ImageNet(mpimg.imread(os.path.join(path,'boneage-test-dataset/'),
                                            transform = val_transform)

# Instantiate loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)

SyntaxError: ignored

In [ ]:
imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)